In [278]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import seaborn as sns
import matplotlib.pyplot as plt

In [279]:
# Định nghĩa lớp CustomRidge
class CustomRidge:
    def __init__(self, alpha=1.0):
        self.alpha = alpha
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        X_with_bias = np.c_[np.ones(X.shape[0]), X]
        n_features = X_with_bias.shape[1]
        I = np.identity(n_features)
        I[0, 0] = 0
        XTX = X_with_bias.T @ X_with_bias
        XTX_reg = XTX + self.alpha * I
        XTy = X_with_bias.T @ y
        self.weights = np.linalg.solve(XTX_reg, XTy)
        self.bias = self.weights[0]
        self.weights = self.weights[1:]

    def predict(self, X):
        return X @ self.weights + self.bias

In [280]:
# 1. Đọc và kiểm tra dữ liệu
df = pd.read_csv('df1.csv') # df1 > df2

In [281]:
# In thông tin cơ bản về dữ liệu
print("Thông tin dữ liệu:")
print(df.info())
print("\nMô tả dữ liệu:")
print(df.describe())
print("\nKiểm tra giá trị thiếu:")
print(df.isnull().sum())

Thông tin dữ liệu:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12051 entries, 0 to 12050
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Location       12051 non-null  object 
 1   Price          12051 non-null  int64  
 2   Type of House  12051 non-null  object 
 3   Land Area      12051 non-null  float64
 4   Bedrooms       12051 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 470.9+ KB
None

Mô tả dữ liệu:
              Price     Land Area      Bedrooms
count  1.205100e+04  12051.000000  12051.000000
mean   6.858117e+09     68.554726      5.747573
std    5.245329e+09     38.604128      8.049289
min    5.200000e+08     10.000000      1.000000
25%    3.390000e+09     42.000000      3.000000
50%    7.500000e+09     64.000000      4.000000
75%    8.000000e+09     73.000000      4.000000
max    2.200000e+11    840.000000     38.000000

Kiểm tra giá trị thiếu:
Location         0
Pric

In [282]:
# 2. Xử lý giá trị thiếu
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())

In [283]:
# 3. Xử lý ngoại lai và giới hạn giá trị bất thường
for col in ['Price', 'Bedrooms', 'Land Area']:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    df = df[~((df[col] < (Q1 - 1.5 * IQR)) | (df[col] > (Q3 + 1.5 * IQR)))]
# Giới hạn Bedrooms <= 10 và Land Area <= 200
df = df[df['Bedrooms'] <= 10]
df = df[df['Land Area'] <= 200]
print(f"Số mẫu sau khi xử lý ngoại lai và giới hạn: {len(df)}")

Số mẫu sau khi xử lý ngoại lai và giới hạn: 10948


In [284]:
# 4. Chuyển đổi biến mục tiêu (Price) sang log scale
df['Price'] = np.log1p(df['Price'])

In [285]:
# 5. Xử lý biến dạng chuỗi: Location, Type of House
df = pd.get_dummies(df, columns=['Location', 'Type of House'], drop_first=True)

In [286]:
# 6. Xác định đặc trưng và biến mục tiêu
X = df.drop('Price', axis=1).to_numpy()
y = df['Price'].to_numpy()

In [287]:
# 7. Chuẩn hóa đặc trưng
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print(f"Số đặc trưng sau mã hóa one-hot: {X_scaled.shape[1]}")

Số đặc trưng sau mã hóa one-hot: 173


In [288]:
# 8. Chia dữ liệu thành train/test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [289]:
# 9. Tinh chỉnh alpha trên tập validation
print("\nTinh chỉnh alpha trên tập validation:")
X_train_sub, X_val, y_train_sub, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
best_alpha = 1.0
best_r2 = -float('inf')
for alpha in [0.1, 1.0, 10.0]:
    ridge = CustomRidge(alpha=alpha)
    ridge.fit(X_train_sub, y_train_sub)
    y_pred_val = ridge.predict(X_val)
    r2_val = r2_score(y_val, y_pred_val)
    print(f"alpha={alpha}, R² trên validation={r2_val:.4f}")
    if r2_val > best_r2:
        best_r2 = r2_val
        best_alpha = alpha
print(f"Giá trị alpha tối ưu: {best_alpha}")


Tinh chỉnh alpha trên tập validation:
alpha=0.1, R² trên validation=0.9111
alpha=1.0, R² trên validation=0.9317
alpha=10.0, R² trên validation=0.9367
Giá trị alpha tối ưu: 10.0


In [290]:
# 10. Huấn luyện và đánh giá mô hình cuối cùng
print("\nTiến hành đánh giá mô hình Ridge Regression")
ridge_custom = CustomRidge(alpha=best_alpha)
ridge_custom.fit(X_train, y_train)
y_pred_custom = ridge_custom.predict(X_test)


Tiến hành đánh giá mô hình Ridge Regression


In [291]:
# Kiểm tra overfitting
y_pred_train = ridge_custom.predict(X_train)
r2_train = r2_score(y_train, y_pred_train)
print(f"R² trên tập train: {r2_train:.4f}")

R² trên tập train: 0.9788


In [292]:
# Tính toán các chỉ số đánh giá trên giá trị log-transformed
mae_custom = mean_absolute_error(y_test, y_pred_custom)
rmse_custom = np.sqrt(mean_squared_error(y_test, y_pred_custom))
r2_custom = r2_score(y_test, y_pred_custom)

print(f"Mean Absolute Error (MAE): {mae_custom:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse_custom:.4f}")
print(f"R² Score trên tập test: {r2_custom:.4f}")

Mean Absolute Error (MAE): 0.0250
Root Mean Squared Error (RMSE): 0.1426
R² Score trên tập test: 0.9385


In [293]:
# 11. Chuyển đổi ngược log để tính toán giá trị thực
y_test_real = np.expm1(y_test)
y_pred_real = np.expm1(y_pred_custom)

mae_real = int(mean_absolute_error(y_test_real, y_pred_real) + 0.5)
rmse_real = int(np.sqrt(mean_squared_error(y_test_real, y_pred_real)) + 0.5)
r2_real = r2_score(y_test_real, y_pred_real)

print("\nKết quả trên giá trị gốc (không log):")
print(f"Mean Absolute Error (MAE): {mae_real:,}")
print(f"Root Mean Squared Error (RMSE): {rmse_real:,}")
print(f"R² Score: {r2_real:.4f}")


Kết quả trên giá trị gốc (không log):
Mean Absolute Error (MAE): 121,003,655
Root Mean Squared Error (RMSE): 622,265,996
R² Score: 0.9438


In [294]:
# 12. Vẽ biểu đồ phân phối Price và giá thực tế vs dự đoán
plt.figure(figsize=(13, 7))
sns.histplot(df['Price'], kde=True)
plt.title('Phân phối của Price (sau log-transform)')
plt.savefig('price_distribution.png')
# plt.show()
plt.close()

In [295]:
plt.figure(figsize=(13, 7))
plt.scatter(y_test_real, y_pred_real, alpha=0.5)
plt.plot([y_test_real.min(), y_test_real.max()], [y_test_real.min(), y_test_real.max()], 'r--')
plt.xlabel('Giá thực tế (VND)')
plt.ylabel('Giá dự đoán (VND)')
plt.title('Giá thực tế vs Giá dự đoán')
plt.savefig('prediction_scatter.png')
# plt.show()
plt.close()